In [1]:
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat

In [3]:
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [4]:
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])
    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [5]:
df_2018 = pd.read_spss("../data/AllbusData/ZA5270_v2-0-0.sav")
df_2018 = germanElectorate(df_2018)
df_2018= reformgroups(df_2018)
df_2018.Alter.unique()

array(['50-64', 'U30', '30-49', 'Ü65'], dtype=object)

In [6]:
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
df_2021 = germanElectorate(df_2021)
df_2021= reformgroups(df_2021)

In [7]:
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
df_2023 = germanElectorate(df_2023)
df_2023= reformgroups(df_2023)


In [8]:
for i in df_2023["land"].unique():
    print(i)

RHEINLAND-PFALZ
BAYERN
THUERINGEN
SCHLESWIG-HOLSTEIN
BADEN-WUERTTEMBERG
NIEDERSACHSEN
BRANDENBURG
NORDRHEIN-WESTFALEN
SACHSEN
HESSEN
EHEM. BERLIN-OST
EHEM. BERLIN-WEST
MECKLENB.-VORPOMMERN
SACHSEN-ANHALT
HAMBURG
SAARLAND
BREMEN


In [9]:
factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]

In [10]:
def analyze_factor_patterns(dataset,variable_to_check):
    base_df=pd.DataFrame()
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #print(element2)
            baselist=[variable_to_check,"za_nr"]
            extralist=[element,element2]
            extralist=list(set(extralist))
    
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]
            if len(dataset_reduced)<50:
                print("low")
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            grouped_df=grouped_df.dropna(axis=1)
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')

            
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    base_df=base_df.drop_duplicates()
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'za_nr','percent']
    
    return base_df[base_df_cols]

In [11]:
df_2023_inflation=analyze_factor_patterns(df_2023,"va03")
df_2023_inflation

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,342,26.677067
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,425,33.151326
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,213,16.614665
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,302,23.556942
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,396,28.989751
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,292,21.581670
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,613,27.366071
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,895,39.955357
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,260,11.607143


In [12]:
df_2021_inflation=analyze_factor_patterns(df_2021,"va03")
df_2021_inflation

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,223,26.143025
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,405,47.479484
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,74,8.675264
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,151,17.702227
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,242,24.007937
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,163,16.979167
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,376,25.066667
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,837,55.800000
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,66,4.400000


In [13]:
comp_inflation=pd.merge(df_2021_inflation,df_2023_inflation,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'va03'],suffixes=["_2021","_2023"])
comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr_2021,percent_2021,za_nr_2023,percent_2023
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,223,26.143025,342,26.677067
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,405,47.479484,425,33.151326
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,74,8.675264,213,16.614665
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,151,17.702227,302,23.556942
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,242,24.007937,396,28.989751
...,...,...,...,...,...,...,...,...,...
255,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,163,16.979167,292,21.581670
256,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,376,25.066667,613,27.366071
257,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,837,55.800000,895,39.955357
258,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,66,4.400000,260,11.607143


In [14]:
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,za_nr,percent
0,30-49,NaN,NaN,NaN,AFD,75,7.324219
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,62,6.054688
2,30-49,NaN,NaN,NaN,CDU-CSU,215,20.996094
3,30-49,NaN,NaN,NaN,DIE GRUENEN,306,29.882812
4,30-49,NaN,NaN,NaN,DIE LINKE,69,6.738281
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,432,23.052295
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,145,7.737460
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,230,12.273212
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,296,15.795091


In [15]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_19499/1039247085.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,za_nr,percent
0,30-49,NaN,NaN,NaN,AFD,160,15.717092
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,47,4.616896
2,30-49,NaN,NaN,NaN,CDU-CSU,206,20.235756
3,30-49,NaN,NaN,NaN,DIE GRUENEN,214,21.021611
4,30-49,NaN,NaN,NaN,DIE LINKE,67,6.581532
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,322,17.349138
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,115,6.196121
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,162,8.728448
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,371,19.989224


In [16]:
comp_inflation["diff_percent"]=comp_inflation["percent_2023"]-comp_inflation["percent_2021"]
comp_inflation[comp_inflation["va03"]=="AM WICHTIGSTEN"].sort_values(by="diff_percent",ascending=False)
#comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr_2021,percent_2021,za_nr_2023,percent_2023,diff_percent
122,U30,dicht besiedelt,NaN,NaN,AM WICHTIGSTEN,7,6.481481,43,22.395833,15.914352
82,U30,NaN,NaN,West,AM WICHTIGSTEN,18,6.870229,89,21.394231,14.524002
26,U30,NaN,Männlich,NaN,AM WICHTIGSTEN,13,7.428571,58,19.931271,12.502700
130,U30,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,11,8.208955,50,20.491803,12.282848
42,U30,NaN,Weiblich,NaN,AM WICHTIGSTEN,19,10.674157,64,22.939068,12.264911
...,...,...,...,...,...,...,...,...,...,...
66,50-64,NaN,NaN,Ost,AM WICHTIGSTEN,13,4.100946,26,6.235012,2.134066
114,50-64,gering besiedelt,NaN,NaN,AM WICHTIGSTEN,15,4.249292,29,6.209850,1.960558
62,50-64,NaN,NaN,Berlin,AM WICHTIGSTEN,2,4.081633,3,5.357143,1.275510
134,Ü65,dicht besiedelt,NaN,NaN,AM WICHTIGSTEN,6,4.054054,11,4.471545,0.417491


In [17]:
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023']].reset_index()

new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,SPD_percent_2021,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023
0,NaN,NaN,NaN,Berlin,5.681818,3.977273,19.318182,28.409091,9.659091,5.113636,20.454545,7.386364,9.793814,4.123711,15.463918,23.195876,20.103093,4.639175,18.041237,4.639175
1,NaN,NaN,NaN,Ost,11.690496,5.298570,23.801514,14.297729,13.204373,11.522288,13.624895,6.560135,21.171171,4.324324,20.900901,10.540541,11.621622,5.945946,19.369369,6.126126
2,NaN,NaN,NaN,West,4.579208,3.547855,27.681518,27.310231,4.867987,12.458746,15.222772,4.331683,8.271605,4.074074,27.572016,22.469136,3.127572,9.465021,20.082305,4.938272
3,NaN,NaN,Männlich,NaN,9.308240,3.916582,25.584944,20.345880,7.985758,13.123093,15.005086,4.730417,14.524556,4.231975,25.496343,16.614420,5.799373,9.247649,18.808777,5.276907
4,NaN,NaN,Männlich,Berlin,8.791209,3.296703,15.384615,26.373626,8.791209,7.692308,21.978022,7.692308,11.111111,5.555556,16.666667,21.111111,18.888889,4.444444,17.777778,4.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,8.452951,1.275917,31.897927,13.237640,7.814992,11.642743,21.690590,3.987241,10.033445,1.170569,34.280936,10.367893,6.020067,6.856187,27.090301,4.180602
61,Ü65,NaN,Weiblich,NaN,2.226345,2.226345,36.920223,15.213358,8.534323,10.018553,20.222635,4.638219,6.560284,2.482270,30.851064,14.539007,5.496454,6.205674,29.078014,4.787234
62,Ü65,dicht besiedelt,NaN,NaN,6.741573,1.685393,30.898876,15.168539,5.617978,8.426966,25.842697,5.617978,7.476636,1.869159,23.364486,10.747664,7.943925,7.943925,36.448598,4.205607
63,Ü65,gering besiedelt,NaN,NaN,6.329114,1.898734,33.544304,13.291139,7.911392,10.759494,20.253165,6.012658,7.619048,2.539683,39.365079,13.015873,3.174603,6.031746,24.444444,3.809524


In [18]:
comp_inflation_pivot=comp_inflation.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='va03')[['percent_2021','percent_2023']].reset_index()
new_columns=[]
for index in comp_inflation_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_inflation_pivot.columns=new_columns
comp_inflation_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,AM WICHTIGSTEN_percent_2023,AM ZWEITWICHTIGSTEN_percent_2023
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,11.013216,23.788546
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,11.560694,21.459538
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,11.517797,20.685024
3,NaN,NaN,Männlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,9.848485,18.850267
4,NaN,NaN,Männlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,8.490566,15.094340
...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,5.829596,18.983558
61,Ü65,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,7.792208,21.500722
62,Ü65,dicht besiedelt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,4.471545,19.105691
63,Ü65,gering besiedelt,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,7.936508,20.899471


In [19]:
comp_inflation_party_pivot=pd.merge(comp_inflation_pivot,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,SPD_percent_2021,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,...,20.454545,7.386364,9.793814,4.123711,15.463918,23.195876,20.103093,4.639175,18.041237,4.639175
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,13.624895,6.560135,21.171171,4.324324,20.900901,10.540541,11.621622,5.945946,19.369369,6.126126
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,15.222772,4.331683,8.271605,4.074074,27.572016,22.469136,3.127572,9.465021,20.082305,4.938272
3,NaN,NaN,Männlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,15.005086,4.730417,14.524556,4.231975,25.496343,16.614420,5.799373,9.247649,18.808777,5.276907
4,NaN,NaN,Männlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,...,21.978022,7.692308,11.111111,5.555556,16.666667,21.111111,18.888889,4.444444,17.777778,4.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,...,21.690590,3.987241,10.033445,1.170569,34.280936,10.367893,6.020067,6.856187,27.090301,4.180602
61,Ü65,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,...,20.222635,4.638219,6.560284,2.482270,30.851064,14.539007,5.496454,6.205674,29.078014,4.787234
62,Ü65,dicht besiedelt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,...,25.842697,5.617978,7.476636,1.869159,23.364486,10.747664,7.943925,7.943925,36.448598,4.205607
63,Ü65,gering besiedelt,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,...,20.253165,6.012658,7.619048,2.539683,39.365079,13.015873,3.174603,6.031746,24.444444,3.809524


In [20]:
zensus=pd.read_csv("json_none.csv")
zensus

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Berlin,U30,0.662670
1,NaN,NaN,Berlin,50-64,1.008255
2,NaN,NaN,Berlin,Ü65,1.054885
3,NaN,NaN,Berlin,30-49,1.264092
4,Männlich,NaN,Berlin,NaN,1.910200
...,...,...,...,...,...
58,Weiblich,NaN,West,NaN,41.055375
59,NaN,mittlere Besiedlungsdichte,NaN,NaN,44.163973
60,Männlich,NaN,NaN,NaN,48.323387
61,Weiblich,NaN,NaN,NaN,51.619587


In [21]:
comp_inflation_party_pivot_zensus=pd.merge(comp_inflation_party_pivot,zensus,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot_zensus

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,...,7.386364,9.793814,4.123711,15.463918,23.195876,20.103093,4.639175,18.041237,4.639175,3.989901
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,6.560135,21.171171,4.324324,20.900901,10.540541,11.621622,5.945946,19.369369,6.126126,16.433332
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,4.331683,8.271605,4.074074,27.572016,22.469136,3.127572,9.465021,20.082305,4.938272,79.519741
3,NaN,NaN,Männlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,4.730417,14.524556,4.231975,25.496343,16.614420,5.799373,9.247649,18.808777,5.276907,48.323387
4,NaN,NaN,Männlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,...,7.692308,11.111111,5.555556,16.666667,21.111111,18.888889,4.444444,17.777778,4.444444,1.910200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,...,3.987241,10.033445,1.170569,34.280936,10.367893,6.020067,6.856187,27.090301,4.180602,12.463294
59,Ü65,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,...,4.638219,6.560284,2.482270,30.851064,14.539007,5.496454,6.205674,29.078014,4.787234,15.997430
60,Ü65,dicht besiedelt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,...,5.617978,7.476636,1.869159,23.364486,10.747664,7.943925,7.943925,36.448598,4.205607,9.247415
61,Ü65,gering besiedelt,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,...,6.012658,7.619048,2.539683,39.365079,13.015873,3.174603,6.031746,24.444444,3.809524,6.025729


In [22]:
comp_inflation_party_pivot_zensus

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,...,7.386364,9.793814,4.123711,15.463918,23.195876,20.103093,4.639175,18.041237,4.639175,3.989901
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,6.560135,21.171171,4.324324,20.900901,10.540541,11.621622,5.945946,19.369369,6.126126,16.433332
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,4.331683,8.271605,4.074074,27.572016,22.469136,3.127572,9.465021,20.082305,4.938272,79.519741
3,NaN,NaN,Männlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,4.730417,14.524556,4.231975,25.496343,16.614420,5.799373,9.247649,18.808777,5.276907,48.323387
4,NaN,NaN,Männlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,...,7.692308,11.111111,5.555556,16.666667,21.111111,18.888889,4.444444,17.777778,4.444444,1.910200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,...,3.987241,10.033445,1.170569,34.280936,10.367893,6.020067,6.856187,27.090301,4.180602,12.463294
59,Ü65,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,...,4.638219,6.560284,2.482270,30.851064,14.539007,5.496454,6.205674,29.078014,4.787234,15.997430
60,Ü65,dicht besiedelt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,...,5.617978,7.476636,1.869159,23.364486,10.747664,7.943925,7.943925,36.448598,4.205607,9.247415
61,Ü65,gering besiedelt,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,...,6.012658,7.619048,2.539683,39.365079,13.015873,3.174603,6.031746,24.444444,3.809524,6.025729


In [25]:
result = comp_inflation_party_pivot_zensus.to_json(orient="records")
parsed = loads(result)
import json

with open('data_for_viz.json', 'w', encoding='utf-8') as f:
    json.dump(parsed, f)